## 0617 과제 - pima indian 데이터셋 분류

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# 변수 설명
# 1. Number of times pregnant
# 2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
# 3. Diastolic blood pressure (mm Hg)
# 4. Triceps skin fold thickness (mm)
# 5. 2-Hour serum insulin (mu U/ml)
# 6. Body mass index (weight in kg/(height in m)^2)
# 7. Diabetes pedigree function
# 8. Age (years)
# 9. Class variable (0 or 1)

In [2]:
# 데이터 불러오기
pima= pd.read_csv('pima-indians-diabetes.csv',header=None)
pima.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# 컬럼명 생성하기
pima.columns = ["pregnant", "pgc", "bp","skin","insulin","bodymass","dpf","age","class"]

In [4]:
pima.head() # 확인

,pregnant,pgc,bp,skin,insulin,bodymass,dpf,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
pima.iloc[:,:8] # 독립변수 뽑는 방법1

,pregnant,pgc,bp,skin,insulin,bodymass,dpf,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [5]:
pima_data = pima.loc[:, pima.columns != "class"]  #독립변수 뽑는 방법 2
pima_data.head()

,pregnant,pgc,bp,skin,insulin,bodymass,dpf,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [6]:
pima_data2 = pima_data.to_numpy() # 독립변수 array 버전

In [7]:
pima_target = pima['class'] # 종속변수
pima_target.head()

0    1
1    0
2    1
3    0
4    1
Name: class, dtype: int64

In [11]:
type(pima['class'])

pandas.core.series.Series

In [12]:
pima['class'].value_counts()

0    500
1    268
Name: class, dtype: int64

## 1. Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
# train/test set 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pima_data, pima_target, stratify = pima_target, test_size=0.2,random_state=2021)

In [15]:
dt_clf = DecisionTreeClassifier(random_state=2020)

In [16]:
# 학습 수행
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=2020)

In [17]:
# 예측 수행
pred_dt = dt_clf.predict(X_test)

In [18]:
# 모델 평가
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred_dt)
print(f'결정트리 예측 정확도{score:.4f}')

결정트리 예측 정확도0.6818


In [19]:
# 교차검증방식
from sklearn.model_selection import cross_val_score

In [20]:
cross_val_score(DecisionTreeClassifier(), pima_data, pima_target, cv=5)

array([0.66883117, 0.65584416, 0.7012987 , 0.77777778, 0.7254902 ])

- 하이퍼파라미터 튜닝

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
dtc = DecisionTreeClassifier(random_state=2010)

In [23]:
# parameter 를 Dictionary 형태로 설정
params = {
    'max_depth':[2,3,4,5,6],
    'min_samples_split':[2,3,4]
}

In [24]:
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=5)  # cv: 교차검증세트 수

In [25]:
grid_dtc.fit(X_train,y_train)  # 학습 수행

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2010),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]})

In [26]:
# 최적 파라미터와 최고 정확도
grid_dtc.best_params_ , grid_dtc.best_score_

({'max_depth': 2, 'min_samples_split': 2}, 0.734572837531654)

In [27]:
# 최적 예측 
pred = grid_dtc.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.7337662337662337

### - 실제 값 하나를 주고 예측

In [29]:
estimator = grid_dtc.best_estimator_
test_data = [4, 110, 92, 0, 0, 37.6, 0.191, 30]
predict = estimator.predict(np.array(test_data).reshape(1,8))

In [30]:
predict

array([0], dtype=int64)

## 2. SVM

In [39]:
from sklearn.svm import SVC

In [104]:
# 학습, 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(pima_data, pima_target, stratify = pima_target, test_size=0.2,random_state=2021)

In [105]:
sv_clf = SVC()  # 객체 생성
sv_clf.fit(X_train,y_train)  #학습 수행

SVC()

In [106]:
# 예측 수행
pred_sv = sv_clf.predict(X_test)

In [107]:
# 모델 평가
score = accuracy_score(y_test, pred_sv)
print(f'SVM 예측 정확도{score:.4f}')

SVM 예측 정확도0.7403


In [108]:
# 교차검증방식
cross_val_score(SVC(), pima_data, pima['class'], cv=5)

array([0.74675325, 0.73376623, 0.77777778, 0.78431373, 0.75163399])

- 최적화

In [137]:
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [138]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [139]:
pipe_svc = Pipeline([('scl', StandardScaler()), ('clf', SVC(random_state=1))])

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [
    {'clf__C': param_range, 'clf__kernel': ['linear']},
    {'clf__C': param_range, 'clf__gamma': param_range, 'clf__kernel': ['rbf']}]

grid_svc  = GridSearchCV(estimator=pipe_svc, param_grid=param_grid,
                  scoring='accuracy', cv=10, n_jobs=1)

In [142]:
# 학습 수행
grid_svc.fit(X_train,y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', SVC(random_state=1))]),
             n_jobs=1,
             param_grid=[{'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                     1000.0],
                          'clf__kernel': ['linear']},
                         {'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                     1000.0],
                          'clf__gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0,
                                         100.0, 1000.0],
                          'clf__kernel': ['rbf']}],
             scoring='accuracy')

In [145]:
# 최적 파라미터와 최고 정확도
print(grid_svc.best_params_)
print(grid_svc.best_score_)

{'clf__C': 100.0, 'clf__gamma': 0.001, 'clf__kernel': 'rbf'}
0.7736118455843469


In [144]:
# 최적 예측 
pred = grid_svc.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.7792207792207793

## 3. Logistic Regression

In [50]:
from sklearn.linear_model import LogisticRegression

In [149]:
# 스케일링
scaled_pima_data = StandardScaler().fit_transform(pima_data)

In [150]:
# 학습, 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(scaled_pima_data, pima_target, stratify=pima_target, test_size=0.2)

In [151]:
lr_clf = LogisticRegression(random_state=2021) # 객체 생성
lr_clf.fit(X_train,y_train)  # 학습 수행

LogisticRegression(random_state=2021)

In [152]:
# 예측 수행
pred_lr = lr_clf.predict(X_test)

In [153]:
# 모델 평가
score = accuracy_score(y_test, pred_lr)
print(f'로지스틱회귀 예측 정확도{score:.4f}')

로지스틱회귀 예측 정확도0.7987


In [155]:
# 교차검증방식
cross_val_score(LogisticRegression(), scaled_pima_data, pima_target, cv=5)

array([0.77272727, 0.74675325, 0.75816993, 0.81045752, 0.77124183])

- 하이퍼파라미터 튜닝

In [159]:
params = { "penalty" : ["l2", "l1"], "C" : [0.01, 0.1, 1, 5, 10] } 
lr = LogisticRegression() 
grid_lr = GridSearchCV(lr, param_grid=params, cv=5, n_jobs=-1, scoring="accuracy")
grid_lr.fit(scaled_pima_data, pima_target)


C:\Users\kimee\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.7627451         nan 0.76664969        nan 0.77186996        nan
 0.76926407        nan 0.76926407        nan]
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 5, 10], 'penalty': ['l2', 'l1']},
             scoring='accuracy')

In [160]:
# 최적 파라미터와 최고 정확도
print("best param : {}".format(grid_lr.best_params_)) 
print("best acc : {}".format(grid_lr.best_score_))

best param : {'C': 1, 'penalty': 'l2'}
best acc : 0.7718699601052542


In [161]:
# 최적 예측 
pred = grid_lr.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.7987012987012987